***RAG Using Faiss***

In [2]:
pip install faiss-cpu sentence-transformers transformers scikit-learn pymupdf numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 70.6 MB/s eta 0:00:00


In [3]:
import os
import time
import fitz  # PyMuPDF for PDF text extraction
import faiss  # Faiss for vector search
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    """Extracts all text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Step 2: Load Embedding and QA Models
embedder = SentenceTransformer('all-MiniLM-L6-v2')
qa_model = pipeline('text2text-generation', model='google/flan-t5-small')

# Step 3: Initialize Faiss Index
d = embedder.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(d)  # L2 distance-based index
documents = []
metadata = []

# Step 4: Embed and Store Text Data into Faiss
def index_pdf_documents(pdf_paths):
    global index, documents, metadata

    for pdf_path in pdf_paths:
        text = extract_text_from_pdf(pdf_path)
        lines = [line.strip() for line in text.splitlines() if line.strip()]

        for i, line in enumerate(lines):
            embedding = embedder.encode([line])[0]
            index.add(np.array([embedding], dtype=np.float32))
            documents.append(line)
            metadata.append({"source": pdf_path, "line_number": i})

# Provide paths to your PDF files
pdf_paths = ["/content/stabale.pdf"]  # Replace with your file path
index_pdf_documents(pdf_paths)

# Step 5: Query Faiss with Cosine Similarity and Threshold
def query_faiss_with_similarity(query, top_k=15, threshold=0.65):
    query_embedding = embedder.encode([query])[0]
    query_embedding = np.array([query_embedding], dtype=np.float32)

    D, I = index.search(query_embedding, top_k)  # Search for top_k nearest neighbors
    similarities = 1 - (D[0] / np.max(D[0])) if np.max(D[0]) != 0 else D[0]  # Convert L2 to cosine-like scores

    filtered_results = [
        (documents[i], similarities[j], metadata[i]['source'])
        for j, i in enumerate(I[0]) if similarities[j] >= threshold
    ]

    return filtered_results, similarities

# Step 6: Generate Context-Augmented Answers
def generate_answer_with_similarity(query, threshold=0.65):
    filtered_results, similarities = query_faiss_with_similarity(query, threshold=threshold)

    print(f"\nQuery: {query}")
    if not filtered_results:
        print("No relevant results found (Below Threshold).")
        return "No relevant results found (Below Threshold).", None, None

    top_result = max(filtered_results, key=lambda x: x[1])
    top_doc, top_similarity, top_source = top_result

    print("Top Relevant Result:")
    print(f"  - Document: {top_doc}")
    print(f"  - Similarity: {top_similarity:.4f}")
    print(f"  - Source: {top_source}")

    context = " ".join([f"Context: {doc}" for doc, _, _ in filtered_results])
    augmented_query = f"Context: {context} Query: {query}"
    response = qa_model(augmented_query)

    return response[0]['generated_text'], top_result, similarities

# Step 7: Evaluate Performance
def evaluate_performance_with_similarity(queries, true_answers, top_k=5, threshold=0.65):
    query_times = []
    correct = 0
    top_cosine_similarity_scores = []

    print("\nCosine Similarities for Each Query:")
    print("-" * 50)

    for query, true_answer in zip(queries, true_answers):
        start_time = time.time()
        augmented_answer, top_result, similarities = generate_answer_with_similarity(query, threshold=threshold)
        query_times.append(time.time() - start_time)

        if top_result:
            top_doc, top_similarity, _ = top_result
            top_cosine_similarity_scores.append(top_similarity)
            print(f"Query: {query}")
            print(f"True Answer: {true_answer}")
            print(f"Top Retrieved Answer: {top_doc}")
            print(f"Cosine Similarity: {top_similarity:.4f}")
            print("-" * 50)

            true_answer_embedding = embedder.encode([true_answer])[0]
            top_doc_embedding = embedder.encode([top_doc])[0]
            sim_score = cosine_similarity([true_answer_embedding], [top_doc_embedding])[0][0]
            if sim_score > 0.8:
                correct += 1
        else:
            top_cosine_similarity_scores.append(0.0)

    avg_latency = np.mean(query_times)
    throughput = len(queries) / sum(query_times)
    avg_cosine_similarity = np.mean(top_cosine_similarity_scores)

    return {
        'average_latency': avg_latency,
        'throughput': throughput,
        'average_cosine_similarity': avg_cosine_similarity,
        'accuracy': correct / len(queries),
        'total_queries': len(queries),
    }

# Step 8: Test Queries and Evaluation
test_queries = [
    "What is Stable Diffusion?",
    "Who developed Stable Diffusion?",
    "What are the primary uses of Stable Diffusion?",
    "What is the architecture of Stable Diffusion?",
    "What is the diffusion process in Stable Diffusion?",
    "What is the role of latent space in Stable Diffusion?",
    "What is U-Net in Stable Diffusion?",
    "What is the role of CLIP in Stable Diffusion?",
    "How does the decoding process work in Stable Diffusion?",
    "What are the key components of Stable Diffusion?",
    "What is the purpose of text encoding in Stable Diffusion?",
    "What is the training process of Stable Diffusion?",
    "What are some real-world applications of Stable Diffusion?",
    "How is Stable Diffusion used in gaming and virtual worlds?",
    "How can Stable Diffusion assist visually impaired individuals?",
    "What is the role of Variational Autoencoder (VAE) in Stable Diffusion?",
    "What industries benefit from Stable Diffusion?",
    "How does Stable Diffusion handle conditional inputs?",
    "What is the significance of inpainting and outpainting in Stable Diffusion?",
    "How does Stable Diffusion contribute to creative content generation?"
    "What is Newton's First Law of Motion?",
    "What is the formula for kinetic energy?",
    "What is the speed of light in a vacuum?",
    "What is Ohm’s Law?",
    "What is the difference between mass and weight?",
    "What is the second law of thermodynamics?",
    "What is Schrödinger's cat thought experiment?",
    "What is the unit of electric charge?",
    "What is the law of universal gravitation?",
    "What is the principle of superposition in physics?"
]

true_answers = [
    "Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.",
    "Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.",
    "Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.",
    "Stable Diffusion is based on latent diffusion models and consists of components like the diffusion process, latent space, U-Net architecture, and CLIP for conditional inputs.",
    "The diffusion process involves a forward step adding noise until an image becomes pure noise and a reverse step removing noise to generate meaningful images.",
    "Stable Diffusion operates in a lower-dimensional latent space by encoding input images using a variational autoencoder (VAE).",
    "U-Net is a deep neural network in Stable Diffusion that helps denoise latent representations by predicting and removing noise.",
    "CLIP (Contrastive Language–Image Pretraining) encodes user-provided text into embeddings to guide image generation.",
    "The decoding process involves using a VAE decoder to transform the denoised latent representation back into pixel space, generating a high-resolution image.",
    "The key components of Stable Diffusion include the diffusion process, latent space, U-Net architecture, CLIP text encoder, and the VAE for encoding and decoding.",
    "Text encoding in Stable Diffusion converts user-provided text into embeddings using CLIP to guide the image generation process.",
    "Training involves teaching the model to reverse the noise process using large datasets of paired image-text data and optimizing it using loss functions.",
    "Stable Diffusion is used in creative content generation, advertising, gaming, product design, media, education, and accessibility.",
    "In gaming and virtual worlds, Stable Diffusion generates textures, assets, and environments for video games.",
    "Stable Diffusion assists visually impaired individuals by generating images based on textual descriptions.",
    "The Variational Autoencoder (VAE) in Stable Diffusion compresses images into a lower-dimensional latent space and decodes them back into pixel space.",
    "Industries such as advertising, gaming, media, healthcare, and education benefit from Stable Diffusion.",
    "Stable Diffusion processes conditional inputs like text or images to guide image generation using CLIP and other techniques.",
    "Inpainting and outpainting in Stable Diffusion allow modifying parts of an image or extending an image beyond its original boundaries.",
    "Stable Diffusion enables creative content generation by producing artwork, illustrations, and concept designs based on text prompts."
    "Newton's First Law states that an object at rest stays at rest and an object in motion stays in motion unless acted upon by an external force.",
    "The formula for kinetic energy is KE = (1/2)mv², where m is mass and v is velocity.",
    "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s).",
    "Ohm’s Law states that the voltage across a conductor is directly proportional to the current through it, given by V = IR, where V is voltage, I is current, and R is resistance.",
    "Mass is the amount of matter in an object, measured in kilograms, while weight is the force exerted by gravity on that mass, measured in newtons.",
    "The second law of thermodynamics states that the total entropy of an isolated system always increases over time.",
    "Schrödinger's cat is a thought experiment in quantum mechanics where a cat in a box can be both alive and dead until observed, illustrating quantum superposition.",
    "The unit of electric charge is the coulomb (C).",
    "Newton's Law of Universal Gravitation states that every mass attracts every other mass with a force proportional to the product of their masses and inversely proportional to the square of the distance between them: F = G(m₁m₂)/r².",
    "The principle of superposition states that when two or more waves overlap, the resultant displacement at any point is the sum of the displacements due to each individual wave."
]

# Evaluate Performance
performance_metrics = evaluate_performance_with_similarity(test_queries, true_answers, top_k=5, threshold=0.4)
print("\nPerformance Metrics:")
print(performance_metrics)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu



Cosine Similarities for Each Query:
--------------------------------------------------

Query: What is Stable Diffusion?
Top Relevant Result:
  - Document: Stable Diffusion
  - Similarity: 0.8219
  - Source: /content/stabale.pdf
Query: What is Stable Diffusion?
True Answer: Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.
Top Retrieved Answer: Stable Diffusion
Cosine Similarity: 0.8219
--------------------------------------------------

Query: Who developed Stable Diffusion?
Top Relevant Result:
  - Document: Stable Diffusion
  - Similarity: 0.6689
  - Source: /content/stabale.pdf
Query: Who developed Stable Diffusion?
True Answer: Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.
Top Retrieved Answer: Stable Diffusion
Cosine Similarity: 0.6689
--------------------------------------------------

Query: What are the primary uses of Stable Di

**RAG Using Chroma**

In [1]:
# Install Required Libraries
!pip install chromadb transformers sentence-transformers PyMuPDF

In [2]:
# Import Required Libraries
import os
import time
import fitz  # PyMuPDF for PDF text extraction
from chromadb.config import Settings
from chromadb import Client
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
# Step 1: Extract Text from PDFs
def extract_text_from_pdf(pdf_path):
    """Extracts all text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Step 2: Load Embedding and QA Models
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # SentenceTransformer for embedding generation
qa_model = pipeline('text2text-generation', model='google/flan-t5-small')  # QA Model for generating responses

# Step 3: Initialize ChromaDB
client = Client(Settings(persist_directory="./chroma_db"))  # Initialize ChromaDB with persistence
collection = client.get_or_create_collection("nlp_collection")  # Create or get a collection
# Step 4: Embed and Store Text Data into ChromaDB
def index_pdf_documents(pdf_paths):
    """
    Index PDF documents into ChromaDB.
    Each line of text is embedded and stored in the database.
    """
    for pdf_path in pdf_paths:
        text = extract_text_from_pdf(pdf_path)
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        for i, line in enumerate(lines):
            embedding = embedder.encode([line])
            collection.add(
                documents=[line],
                embeddings=embedding,
                metadatas=[{"source": pdf_path, "line_number": i}],
                ids=[f"{os.path.basename(pdf_path)}_{i}"]
            )

# Provide paths to your PDF files
pdf_paths = ["/content/stabale.pdf"]  # Replace with your file path
index_pdf_documents(pdf_paths)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Step 4: Embed and Store Text Data into ChromaDB
def index_pdf_documents(pdf_paths):
    """
    Index PDF documents into ChromaDB.
    Each line of text is embedded and stored in the database.
    """
    for pdf_path in pdf_paths:
        text = extract_text_from_pdf(pdf_path)
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        for i, line in enumerate(lines):
            embedding = embedder.encode([line])
            collection.add(
                documents=[line],
                embeddings=embedding,
                metadatas=[{"source": pdf_path, "line_number": i}],
                ids=[f"{os.path.basename(pdf_path)}_{i}"]
            )

# Provide paths to your PDF files
pdf_paths = ["/content/stabale.pdf"]  # Replace with your file path
index_pdf_documents(pdf_paths)

In [11]:
# Step 5: Query ChromaDB with Cosine Similarity and Threshold
def query_chromadb_with_similarity(query, top_k=15, threshold=0.65):
    """
    Query ChromaDB for the most relevant text snippets with cosine similarity and threshold filtering.
    """
    query_embedding = embedder.encode([query])  # Generate query embedding
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)
    retrieved_docs = results['documents'][0]
    retrieved_embeddings = embedder.encode(retrieved_docs)

    # Calculate cosine similarity
    similarities = cosine_similarity(query_embedding, retrieved_embeddings)[0]

    # Filter results based on the threshold
    filtered_results = [
        (doc, similarity, meta["source"])
        for doc, similarity, meta in zip(retrieved_docs, similarities, results['metadatas'][0])
        if similarity >= threshold
    ]

    return filtered_results, similarities

# Step 6: Generate Context-Augmented Answers with Top Result and Cosine Similarity
def generate_answer_with_similarity(query, threshold=0.65):
    """
    Generate a context-augmented answer using the QA model.
    Retrieves the most relevant result from ChromaDB based on cosine similarity.
    """
    filtered_results, similarities = query_chromadb_with_similarity(query, threshold=threshold)

    print(f"\nQuery: {query}")
    if not filtered_results:
        print("No relevant results found (Below Threshold).")
        return "No relevant results found (Below Threshold).", None, None

    # Retrieve the top result
    top_result = max(filtered_results, key=lambda x: x[1])  # Sort by similarity score
    top_doc, top_similarity, top_source = top_result

    # Print the top result details
    print("Top Relevant Result:")
    print(f"  - Document: {top_doc}")
    print(f"  - Similarity: {top_similarity:.4f}")
    print(f"  - Source: {top_source}")

    # Generate context string from all retrieved results
    context = " ".join([f"Context: {doc}" for doc, _, _ in filtered_results])
    augmented_query = f"Context: {context} Query: {query}"
    response = qa_model(augmented_query)

    return response[0]['generated_text'], top_result, similarities


In [12]:
# Step 7: Evaluate Performance with Top Result Cosine Similarity
def evaluate_performance_with_similarity(queries, true_answers, top_k=5, threshold=0.65):
    """
    Evaluate the performance of the ChromaDB-based retrieval system.
    Metrics include cosine similarity for the top result.
    """
    query_times = []
    correct = 0
    top_cosine_similarity_scores = []

    print("\nCosine Similarities for Each Query:")
    print("-" * 50)

    for query, true_answer in zip(queries, true_answers):
        # Start timer
        start_time = time.time()

        # Query ChromaDB and get the top result
        augmented_answer, top_result, similarities = generate_answer_with_similarity(query, threshold=threshold)

        # Stop timer
        query_times.append(time.time() - start_time)

        # Check if there's a top result
        if top_result:
            top_doc, top_similarity, _ = top_result
            top_cosine_similarity_scores.append(top_similarity)

            # Print the cosine similarity for the top result
            print(f"Query: {query}")
            print(f"True Answer: {true_answer}")
            print(f"Top Retrieved Answer: {top_doc}")
            print(f"Cosine Similarity: {top_similarity:.4f}")
            print("-" * 50)

            # Check correctness (e.g., threshold > 0.8)
            true_answer_embedding = embedder.encode([true_answer])
            top_doc_embedding = embedder.encode([top_doc])
            sim_score = cosine_similarity(true_answer_embedding, top_doc_embedding)[0][0]
            if sim_score > 0.8:
                correct += 1
        else:
            top_cosine_similarity_scores.append(0.0)

    avg_latency = np.mean(query_times)
    throughput = len(queries) / sum(query_times)
    avg_cosine_similarity = np.mean(top_cosine_similarity_scores)

    return {
        'average_latency': avg_latency,
        'throughput': throughput,
        'average_cosine_similarity': avg_cosine_similarity,
        'accuracy': correct / len(queries),
        'total_queries': len(queries),
    }

In [13]:
# Step 8: Test Queries and Evaluation
test_queries = [
    "What is Stable Diffusion?",
    "Who developed Stable Diffusion?",
    "What are the primary uses of Stable Diffusion?",
    "What is the architecture of Stable Diffusion?",
    "What is the diffusion process in Stable Diffusion?",
    "What is the role of latent space in Stable Diffusion?",
    "What is U-Net in Stable Diffusion?",
    "What is the role of CLIP in Stable Diffusion?",
    "How does the decoding process work in Stable Diffusion?",
    "What are the key components of Stable Diffusion?",
    "What is the purpose of text encoding in Stable Diffusion?",
    "What is the training process of Stable Diffusion?",
    "What are some real-world applications of Stable Diffusion?",
    "How is Stable Diffusion used in gaming and virtual worlds?",
    "How can Stable Diffusion assist visually impaired individuals?",
    "What is the role of Variational Autoencoder (VAE) in Stable Diffusion?",
    "What industries benefit from Stable Diffusion?",
    "How does Stable Diffusion handle conditional inputs?",
    "What is the significance of inpainting and outpainting in Stable Diffusion?",
    "How does Stable Diffusion contribute to creative content generation?"
    "What is Newton's First Law of Motion?",
    "What is the formula for kinetic energy?",
    "What is the speed of light in a vacuum?",
    "What is Ohm’s Law?",
    "What is the difference between mass and weight?",
    "What is the second law of thermodynamics?",
    "What is Schrödinger's cat thought experiment?",
    "What is the unit of electric charge?",
    "What is the law of universal gravitation?",
    "What is the principle of superposition in physics?"
]

true_answers = [
    "Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.",
    "Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.",
    "Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.",
    "Stable Diffusion is based on latent diffusion models and consists of components like the diffusion process, latent space, U-Net architecture, and CLIP for conditional inputs.",
    "The diffusion process involves a forward step adding noise until an image becomes pure noise and a reverse step removing noise to generate meaningful images.",
    "Stable Diffusion operates in a lower-dimensional latent space by encoding input images using a variational autoencoder (VAE).",
    "U-Net is a deep neural network in Stable Diffusion that helps denoise latent representations by predicting and removing noise.",
    "CLIP (Contrastive Language–Image Pretraining) encodes user-provided text into embeddings to guide image generation.",
    "The decoding process involves using a VAE decoder to transform the denoised latent representation back into pixel space, generating a high-resolution image.",
    "The key components of Stable Diffusion include the diffusion process, latent space, U-Net architecture, CLIP text encoder, and the VAE for encoding and decoding.",
    "Text encoding in Stable Diffusion converts user-provided text into embeddings using CLIP to guide the image generation process.",
    "Training involves teaching the model to reverse the noise process using large datasets of paired image-text data and optimizing it using loss functions.",
    "Stable Diffusion is used in creative content generation, advertising, gaming, product design, media, education, and accessibility.",
    "In gaming and virtual worlds, Stable Diffusion generates textures, assets, and environments for video games.",
    "Stable Diffusion assists visually impaired individuals by generating images based on textual descriptions.",
    "The Variational Autoencoder (VAE) in Stable Diffusion compresses images into a lower-dimensional latent space and decodes them back into pixel space.",
    "Industries such as advertising, gaming, media, healthcare, and education benefit from Stable Diffusion.",
    "Stable Diffusion processes conditional inputs like text or images to guide image generation using CLIP and other techniques.",
    "Inpainting and outpainting in Stable Diffusion allow modifying parts of an image or extending an image beyond its original boundaries.",
    "Stable Diffusion enables creative content generation by producing artwork, illustrations, and concept designs based on text prompts."
    "Newton's First Law states that an object at rest stays at rest and an object in motion stays in motion unless acted upon by an external force.",
    "The formula for kinetic energy is KE = (1/2)mv², where m is mass and v is velocity.",
    "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s).",
    "Ohm’s Law states that the voltage across a conductor is directly proportional to the current through it, given by V = IR, where V is voltage, I is current, and R is resistance.",
    "Mass is the amount of matter in an object, measured in kilograms, while weight is the force exerted by gravity on that mass, measured in newtons.",
    "The second law of thermodynamics states that the total entropy of an isolated system always increases over time.",
    "Schrödinger's cat is a thought experiment in quantum mechanics where a cat in a box can be both alive and dead until observed, illustrating quantum superposition.",
    "The unit of electric charge is the coulomb (C).",
    "Newton's Law of Universal Gravitation states that every mass attracts every other mass with a force proportional to the product of their masses and inversely proportional to the square of the distance between them: F = G(m₁m₂)/r².",
    "The principle of superposition states that when two or more waves overlap, the resultant displacement at any point is the sum of the displacements due to each individual wave."
]

# Evaluate Performance
performance_metrics = evaluate_performance_with_similarity(test_queries, true_answers, top_k=5, threshold=0.4)

print("\nPerformance Metrics:")
print(performance_metrics)


Cosine Similarities for Each Query:
--------------------------------------------------

Query: What is Stable Diffusion?
Top Relevant Result:
  - Document: Stable Diffusion
  - Similarity: 0.8721
  - Source: /content/stabale.pdf
Query: What is Stable Diffusion?
True Answer: Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.
Top Retrieved Answer: Stable Diffusion
Cosine Similarity: 0.8721
--------------------------------------------------

Query: Who developed Stable Diffusion?
Top Relevant Result:
  - Document: Stable Diffusion
  - Similarity: 0.7452
  - Source: /content/stabale.pdf
Query: Who developed Stable Diffusion?
True Answer: Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.
Top Retrieved Answer: Stable Diffusion
Cosine Similarity: 0.7452
--------------------------------------------------

Query: What are the primary uses of Stable Di

**RAG Using Annoy**

In [4]:
# Install required libraries
!pip install PyPDF2 transformers annoy datasets scikit-learn sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553118 sha256=f45390f64d43ba78ac566c00c8821f4734f3061f4101d01f15654d2cc47d9877
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [16]:
import os
import time
import numpy as np
from annoy import AnnoyIndex
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load Embedding Model and Language Model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # Hugging Face Model
qa_model = pipeline('text2text-generation', model='google/flan-t5-small')  # Hugging Face Model

# Step 2: Parse PDF Documents and Create a Corpus
def parse_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = '/content/stabale.pdf'  # Update with actual PDF file path
pdf_text = parse_pdf(pdf_path)

# Step 3: Extract Q&A Pairs
def extract_qa_pairs(text):
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    questions, answers = [], []
    for i, line in enumerate(lines):
        if i % 2 == 0:
            questions.append(line)
        else:
            answers.append(line)
    return questions, answers

questions, answers = extract_qa_pairs(pdf_text)

# Step 4: Generate Embeddings
embeddings = embedding_model.encode(questions)

# Step 5: Create Annoy Index
dimension = embeddings.shape[1]
annoy_index = AnnoyIndex(dimension, 'angular')

for i, emb in enumerate(embeddings):
    annoy_index.add_item(i, emb)

annoy_index.build(10)  # Build with 10 trees

# Step 6: Query the Vector Database with Cosine Similarity
def query_annoy_with_similarity(question, top_k=5):
    question_embedding = embedding_model.encode([question])[0]
    indices = annoy_index.get_nns_by_vector(question_embedding, top_k, include_distances=False)

    # Calculate cosine similarity for each retrieved question
    similarities = [
        cosine_similarity([question_embedding], [embeddings[idx]])[0][0]
        for idx in indices
    ]

    # Filter results
    valid_results = [
        (questions[idx], answers[idx], similarities[i])
        for i, idx in enumerate(indices)
        if idx < len(questions) and idx < len(answers)
    ]

    # Sort by similarity (highest first)
    valid_results.sort(key=lambda x: x[2], reverse=True)

    return valid_results

# Step 7: Compute Cosine Similarity between True and Retrieved Answers
def compute_cosine_similarity(true_answer, retrieved_answer):
    true_emb = embedding_model.encode([true_answer])[0]
    retrieved_emb = embedding_model.encode([retrieved_answer])[0]
    return cosine_similarity([true_emb], [retrieved_emb])[0][0]

# Step 8: Evaluate Performance and Similarity
def evaluate_performance_and_similarity(test_queries, true_answers):
    query_times = []
    results = []
    correct = 0

    for query, true_answer in zip(test_queries, true_answers):
        start_time = time.time()
        retrieved_results = query_annoy_with_similarity(query)
        query_times.append(time.time() - start_time)

        if retrieved_results:
            retrieved_answer = retrieved_results[0][1]  # Best retrieved answer
            similarity_score = compute_cosine_similarity(true_answer, retrieved_answer)
        else:
            retrieved_answer = "No retrieved answer"
            similarity_score = 0.0

        # Store results
        results.append({
            "true_query": query,
            "true_answer": true_answer,
            "retrieved_answer": retrieved_answer,
            "cosine_similarity": similarity_score
        })

        # Check correctness
        if true_answer.lower() in retrieved_answer.lower():
            correct += 1

    # Compute performance metrics
    avg_latency = np.mean(query_times)
    throughput = len(test_queries) / sum(query_times)
    accuracy = correct / len(test_queries)

    performance_metrics = {
        'average_latency': avg_latency,
        'throughput': throughput,
        'accuracy': accuracy,
        'total_queries': len(test_queries)
    }

    return results, performance_metrics

# Step 9: Define Test Queries
test_queries = [
    "What is Stable Diffusion?",
    "Who developed Stable Diffusion?",
    "What are the primary uses of Stable Diffusion?",
    "What is the architecture of Stable Diffusion?",
    "What is the diffusion process in Stable Diffusion?",
    "What is the role of latent space in Stable Diffusion?",
    "What is U-Net in Stable Diffusion?",
    "What is the role of CLIP in Stable Diffusion?",
    "How does the decoding process work in Stable Diffusion?",
    "What are the key components of Stable Diffusion?",
    "What is the purpose of text encoding in Stable Diffusion?",
    "What is the training process of Stable Diffusion?",
    "What are some real-world applications of Stable Diffusion?",
    "How is Stable Diffusion used in gaming and virtual worlds?",
    "How can Stable Diffusion assist visually impaired individuals?",
    "What is the role of Variational Autoencoder (VAE) in Stable Diffusion?",
    "What industries benefit from Stable Diffusion?",
    "How does Stable Diffusion handle conditional inputs?",
    "What is the significance of inpainting and outpainting in Stable Diffusion?",
    "How does Stable Diffusion contribute to creative content generation?"
    "What is Newton's First Law of Motion?",
    "What is the formula for kinetic energy?",
    "What is the speed of light in a vacuum?",
    "What is Ohm’s Law?",
    "What is the difference between mass and weight?",
    "What is the second law of thermodynamics?",
    "What is Schrödinger's cat thought experiment?",
    "What is the unit of electric charge?",
    "What is the law of universal gravitation?",
    "What is the principle of superposition in physics?"
]

true_answers = [
    "Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.",
    "Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.",
    "Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.",
    "Stable Diffusion is based on latent diffusion models and consists of components like the diffusion process, latent space, U-Net architecture, and CLIP for conditional inputs.",
    "The diffusion process involves a forward step adding noise until an image becomes pure noise and a reverse step removing noise to generate meaningful images.",
    "Stable Diffusion operates in a lower-dimensional latent space by encoding input images using a variational autoencoder (VAE).",
    "U-Net is a deep neural network in Stable Diffusion that helps denoise latent representations by predicting and removing noise.",
    "CLIP (Contrastive Language–Image Pretraining) encodes user-provided text into embeddings to guide image generation.",
    "The decoding process involves using a VAE decoder to transform the denoised latent representation back into pixel space, generating a high-resolution image.",
    "The key components of Stable Diffusion include the diffusion process, latent space, U-Net architecture, CLIP text encoder, and the VAE for encoding and decoding.",
    "Text encoding in Stable Diffusion converts user-provided text into embeddings using CLIP to guide the image generation process.",
    "Training involves teaching the model to reverse the noise process using large datasets of paired image-text data and optimizing it using loss functions.",
    "Stable Diffusion is used in creative content generation, advertising, gaming, product design, media, education, and accessibility.",
    "In gaming and virtual worlds, Stable Diffusion generates textures, assets, and environments for video games.",
    "Stable Diffusion assists visually impaired individuals by generating images based on textual descriptions.",
    "The Variational Autoencoder (VAE) in Stable Diffusion compresses images into a lower-dimensional latent space and decodes them back into pixel space.",
    "Industries such as advertising, gaming, media, healthcare, and education benefit from Stable Diffusion.",
    "Stable Diffusion processes conditional inputs like text or images to guide image generation using CLIP and other techniques.",
    "Inpainting and outpainting in Stable Diffusion allow modifying parts of an image or extending an image beyond its original boundaries.",
    "Stable Diffusion enables creative content generation by producing artwork, illustrations, and concept designs based on text prompts."
    "Newton's First Law states that an object at rest stays at rest and an object in motion stays in motion unless acted upon by an external force.",
    "The formula for kinetic energy is KE = (1/2)mv², where m is mass and v is velocity.",
    "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s).",
    "Ohm’s Law states that the voltage across a conductor is directly proportional to the current through it, given by V = IR, where V is voltage, I is current, and R is resistance.",
    "Mass is the amount of matter in an object, measured in kilograms, while weight is the force exerted by gravity on that mass, measured in newtons.",
    "The second law of thermodynamics states that the total entropy of an isolated system always increases over time.",
    "Schrödinger's cat is a thought experiment in quantum mechanics where a cat in a box can be both alive and dead until observed, illustrating quantum superposition.",
    "The unit of electric charge is the coulomb (C).",
    "Newton's Law of Universal Gravitation states that every mass attracts every other mass with a force proportional to the product of their masses and inversely proportional to the square of the distance between them: F = G(m₁m₂)/r².",
    "The principle of superposition states that when two or more waves overlap, the resultant displacement at any point is the sum of the displacements due to each individual wave."
]

# Step 10: Run Evaluation
evaluation_results, performance_metrics = evaluate_performance_and_similarity(test_queries, true_answers)

# Print Performance Metrics
print("\nPerformance Metrics:")
print(performance_metrics)

# Print Evaluation Results
print("\nEvaluation Results:")
for result in evaluation_results:
    print(f"Query: {result['true_query']}")
    print(f"True Answer: {result['true_answer']}")
    print(f"Retrieved Answer: {result['retrieved_answer']}")
    print(f"Cosine Similarity: {result['cosine_similarity']:.4f}\n")


Device set to use cpu



Performance Metrics:
{'average_latency': 0.04684653775445346, 'throughput': 21.346294687592682, 'accuracy': 0.0, 'total_queries': 29}

Evaluation Results:
Query: What is Stable Diffusion?
True Answer: Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.
Retrieved Answer:  Stable Diffusion  is a deep learning , text-to-image model  released
Cosine Similarity: 0.9567

Query: Who developed Stable Diffusion?
True Answer: Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.
Retrieved Answer:  Stable Diffusion  is a deep learning , text-to-image model  released
Cosine Similarity: 0.4866

Query: What are the primary uses of Stable Diffusion?
True Answer: Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.
Retrieved Answer:  Stable Diffusion  is a de

**RAG Using Milvus**

In [ ]:
!pip install pymilvus langchain-community sentence-transformers pdfplumber torch transformers
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
# Import libraries
import time
import json
from numpy import dot
from numpy.linalg import norm
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PDFPlumberLoader
from transformers import pipeline

In [ ]:
# Load the document and embedding model
loader = PDFPlumberLoader("/content/stabale.pdf")
docs = loader.load()
print("Number of pages in the PDF:", len(docs))

# Extract text from each page for embedding
doc_texts = [page.page_content for page in docs]
print("Document texts loaded successfully.")

# Use a free embedding model from Hugging Face
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
sentence_transformer = SentenceTransformer(embedding_model_name)

# Generate embeddings for each page's content
start_time = time.time()
vectors = sentence_transformer.encode(doc_texts)
end_time = time.time()
embedding_latency = (end_time - start_time) / len(doc_texts)
print(f"Average latency for embedding generation: {embedding_latency:.4f} seconds per page")

# Organize the data with ID, vector, text, and a subject label
data = [
    {"id": i, "vector": vector.tolist(), "text": doc_texts[i], "subject": "Data Science"}
    for i, vector in enumerate(vectors)
]

# Create a Milvus client and insert data into a collection
client = MilvusClient("./milvus_local.db")
collection_name = "NLP"
client.create_collection(
    collection_name=collection_name,
    dimension=len(vectors[0]),
    primary_field="id"
)
client.insert(collection_name=collection_name, data=data)
print("Data inserted successfully into Milvus.")

Number of pages in the PDF: 4
Document texts loaded successfully.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average latency for embedding generation: 0.2310 seconds per page
Data inserted successfully into Milvus.


In [ ]:
# Cosine similarity function
def cosine_similarity(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# Adjusted search_data_in_vector_db function
def search_data_in_vector_db(query: str):
    query_vector = sentence_transformer.encode([query])[0].tolist()
    start_time = time.time()
    results = client.search(
        collection_name=collection_name,
        data=[query_vector],
        limit=2,
        output_fields=["text", "subject"]
    )
    end_time = time.time()
    search_latency = end_time - start_time
    print(f"Search latency: {search_latency:.4f} seconds")

    # Check if results are empty or have the expected structure
    if not results or not isinstance(results[0], list) or not results[0] or not isinstance(results[0][0], dict):
        print("Unexpected results structure from Milvus. Check your data and query.")
        return []

    # Calculate cosine similarity for each result and update the result dictionary
    for result in results[0]:
        if "vector" in result:  # Check if 'vector' key exists
            retrieved_vector = result["vector"]
            similarity = cosine_similarity(query_vector, retrieved_vector)
            # Update the result dictionary with cosine similarity
            result["cosine_similarity"] = similarity
        else:
            print(f"Warning: 'vector' key not found in result: {result}")

    return results[0]  # Return the first element of the list, which should be the search results

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity

# Load the sentence transformer model
sentence_transformer = SentenceTransformer("all-MiniLM-L6-v2")

# Define 30 questions and answers
questions = [
    {"id": 1, "text": "What is Stable Diffusion?", "answer": "Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques."},
    {"id": 2, "text": "Who developed Stable Diffusion?", "answer": "Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway."},
    {"id": 3, "text": "What are the primary uses of Stable Diffusion?", "answer": "Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts."},
    {"id": 4, "text": "What is the architecture of Stable Diffusion?", "answer": "Stable Diffusion is based on latent diffusion models and consists of components like the diffusion process, latent space, U-Net architecture, and CLIP for conditional inputs."},
    {"id": 5, "text": "What is the diffusion process in Stable Diffusion?", "answer": "The diffusion process involves a forward step adding noise until an image becomes pure noise and a reverse step removing noise to generate meaningful images."},
    {"id": 6, "text": "What is the role of latent space in Stable Diffusion?", "answer": "Stable Diffusion operates in a lower-dimensional latent space by encoding input images using a variational autoencoder (VAE)."},
    {"id": 7, "text": "What is the function of U-Net in Stable Diffusion?", "answer": "U-Net is a deep neural network in Stable Diffusion that helps denoise latent representations by predicting and removing noise."},
    {"id": 8, "text": "How does CLIP contribute to Stable Diffusion?", "answer": "CLIP (Contrastive Language–Image Pretraining) encodes user-provided text into embeddings to guide image generation."},
    {"id": 9, "text": "What is the role of the VAE in Stable Diffusion?", "answer": "The VAE compresses images into a lower-dimensional latent space and decodes them back into pixel space."},
    {"id": 10, "text": "What is the importance of text encoding in Stable Diffusion?", "answer": "Text encoding converts user-provided text into embeddings using CLIP to guide the image generation process."},
    {"id": 11, "text": "How is Stable Diffusion trained?", "answer": "Training involves teaching the model to reverse the noise process using large datasets of paired image-text data and optimizing it using loss functions."},
    {"id": 12, "text": "What industries benefit from Stable Diffusion?", "answer": "Industries such as advertising, gaming, media, healthcare, and education benefit from Stable Diffusion."},
    {"id": 13, "text": "What is inpainting in Stable Diffusion?", "answer": "Inpainting allows modifying parts of an image while keeping the rest unchanged."},
    {"id": 14, "text": "What is outpainting in Stable Diffusion?", "answer": "Outpainting extends an image beyond its original boundaries."},
    {"id": 15, "text": "How does Stable Diffusion enable creative content generation?", "answer": "Stable Diffusion enables creative content generation by producing artwork, illustrations, and concept designs based on text prompts."},
    {"id": 16, "text": "What is the significance of diffusion sampling in Stable Diffusion?", "answer": "Noise is incrementally added and removed in latent space, and the model iteratively refines latent noise guided by text prompts."},
    {"id": 17, "text": "How does Stable Diffusion assist visually impaired individuals?", "answer": "Stable Diffusion assists visually impaired individuals by generating images based on textual descriptions."},
    {"id": 18, "text": "What is the significance of personalized content in Stable Diffusion?", "answer": "Stable Diffusion enables creating custom avatars, wallpapers, or art for individuals."},
    {"id": 19, "text": "How is Stable Diffusion used in gaming?", "answer": "In gaming, Stable Diffusion generates textures, assets, and environments for video games."},
    {"id": 20, "text": "What is the optimization technique used in Stable Diffusion?", "answer": "Loss functions measure the error in predicting noise to optimize the model."},
    {"id": 21, "text": "How does quantum mechanics contribute to modern technology?", "answer": "Quantum mechanics underpins technologies such as semiconductors, lasers, and quantum computing."},
    {"id": 22, "text": "What role does relativity play in GPS technology?", "answer": "GPS satellites account for time dilation effects from both special and general relativity to maintain accuracy."},
    {"id": 23, "text": "What is the final step in nuclear fission?", "answer": "The nucleus splits into smaller fragments, releasing energy and neutrons that sustain the chain reaction."},
    {"id": 24, "text": "How does superconductivity integrate with technological advancements?", "answer": "Superconductors enable lossless power transmission, MRI machines, and advanced computing applications."},
    {"id": 25, "text": "What is the role of physics in material science?", "answer": "Physics explains material properties, enabling the design of stronger, lighter, and more conductive materials."},
    {"id": 26, "text": "How does physics contribute to renewable energy?", "answer": "Physics principles guide solar panel efficiency, wind turbine aerodynamics, and battery advancements."},
    {"id": 27, "text": "What is the importance of particle accelerators in physics research?", "answer": "Particle accelerators help explore fundamental particles, aiding discoveries in quantum mechanics and cosmology."},
    {"id": 28, "text": "What differentiates classical and quantum mechanics?", "answer": "Classical mechanics describes macroscopic objects, while quantum mechanics governs atomic and subatomic scales."},
    {"id": 29, "text": "How does thermodynamics relate to everyday life?", "answer": "Thermodynamics governs heat transfer, engine efficiency, refrigeration, and biological energy metabolism."},
    {"id": 30, "text": "What is the future potential of quantum computing?", "answer": "Quantum computing promises breakthroughs in cryptography, material simulation, and AI optimization."}
]

# Generate embeddings for each question
for question in questions:
    question_vector = sentence_transformer.encode(question["text"]).reshape(1, -1)
    question["vector"] = question_vector  # Store the embedded vector

# Example query
query = "Explain Stable Diffusion's working principle"
query_vector = sentence_transformer.encode(query).reshape(1, -1)

# Performance Metrics Initialization
total_queries = len(questions)
latencies = []

# Compute cosine similarity and measure latency
start_time = time.time()
for question in questions:
    query_start_time = time.time()
    similarity = cosine_similarity(query_vector, question["vector"])[0][0]
    question["similarity"] = similarity
    query_end_time = time.time()
    latencies.append(query_end_time - query_start_time)
end_time = time.time()

total_time = end_time - start_time
average_latency = np.mean(latencies)
throughput = total_queries / total_time

# Print performance metrics
print(f"Total Queries: {total_queries}")
print(f"Average Latency: {average_latency:.4f} seconds")
print(f"Throughput: {throughput:.2f} queries per second")
print("-" * 40)

# Print results
for question in questions:
    print(f"Question ID: {question['id']}")
    print(f"Question: {question['text']}")
    print(f"Answer: {question['answer']}")
    print(f"Cosine Similarity: {question['similarity']:.4f}")
    print("-" * 40)

Total Queries: 30
Average Latency: 0.0009 seconds
Throughput: 1103.57 queries per second
----------------------------------------
Question ID: 1
Question: What is Stable Diffusion?
Answer: Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.
Cosine Similarity: 0.8073
----------------------------------------
Question ID: 2
Question: Who developed Stable Diffusion?
Answer: Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.
Cosine Similarity: 0.6394
----------------------------------------
Question ID: 3
Question: What are the primary uses of Stable Diffusion?
Answer: Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.
Cosine Similarity: 0.6907
----------------------------------------
Question ID: 4
Question: What is the architecture of Stable Dif

**RAG Using ElasticSearch**

In [17]:
!pip install elasticsearch sentence-transformers transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.0 MB/s eta 0:00:00


In [18]:
!pip install PyPDF2

In [21]:
import os
import time
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader

# Step 1: Connect to Elasticsearch
CLOUD_ID = "newrag:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDVkNGE5MzczOGVhYjQxY2NhNjNiNGFkYzU3Yzc4OGM3JGVhZTIxODczZDNjODRlM2ViNGI5ZmVhYjc0YjNhM2Q3"
API_KEY = "RDJkeUk1VUJubFgyUXJabG54R0w6clpXRE0tVFJTMmF1ZXB6Mnd1ZXNwZw=="

es = Elasticsearch(cloud_id=CLOUD_ID, api_key=API_KEY)

# Test the connection
try:
    print(es.info())  # Get cluster info
    print("✅ Successfully connected to Elasticsearch!")
except Exception as e:
    print(f"❌ Connection failed: {e}")

# Step 2: Define the Index Name
INDEX_NAME = "qa_embeddings"

# Step 3: Create Elasticsearch Index for Embeddings
def create_index():
    if not es.indices.exists(index=INDEX_NAME):
        index_config = {
            "mappings": {
                "properties": {
                    "question": {"type": "text"},
                    "answer": {"type": "text"},
                    "embedding": {"type": "dense_vector", "dims": 384}  # Match embedding dimensions
                }
            }
        }
        es.indices.create(index=INDEX_NAME, body=index_config)

create_index()

# Step 4: Load the Embedding Model and QA Model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
qa_model = pipeline('text2text-generation', model='google/flan-t5-small')

# Step 5: Parse PDF and Create Embeddings
def parse_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def extract_qa_pairs(text):
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    questions, answers = [], []
    for i, line in enumerate(lines):
        if i % 2 == 0:
            questions.append(line)
        else:
            answers.append(line)
    return questions, answers

def index_data(questions, answers):
    documents = []
    for i, (q, a) in enumerate(zip(questions, answers)):
        embedding = embedding_model.encode(q).tolist()
        documents.append({
            "_index": INDEX_NAME,
            "_id": i,
            "_source": {
                "question": q,
                "answer": a,
                "embedding": embedding
            }
        })
    bulk(es, documents)
    print(f"✅ Indexed {len(documents)} Q&A pairs into Elasticsearch!")

# Replace with your PDF path
pdf_path = '/content/stabale.pdf'
pdf_text = parse_pdf(pdf_path)
questions, answers = extract_qa_pairs(pdf_text)
index_data(questions, answers)

# Step 6: Query Elasticsearch Index
def query_elasticsearch_with_similarity(question, top_k=5):
    question_embedding = embedding_model.encode(question).tolist()
    query = {
        "size": top_k,
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": question_embedding}
                }
            }
        }
    }
    response = es.search(index=INDEX_NAME, body=query)
    results = []
    for hit in response['hits']['hits']:
        source = hit["_source"]
        similarity = hit["_score"] - 1.0  # Adjust for the +1.0 in the script
        results.append((source["question"], source["answer"], similarity))
    return results

# Step 7: Compute Cosine Similarity between True and Retrieved Answers
def compute_cosine_similarity(true_answer, retrieved_answer):
    true_emb = embedding_model.encode([true_answer])[0]
    retrieved_emb = embedding_model.encode([retrieved_answer])[0]
    return cosine_similarity([true_emb], [retrieved_emb])[0][0]

# Step 8: Evaluate Performance and Similarity
def evaluate_performance_and_similarity(test_queries, true_answers):
    query_times = []
    results = []
    correct = 0

    for query, true_answer in zip(test_queries, true_answers):
        start_time = time.time()
        retrieved_results = query_elasticsearch_with_similarity(query)
        query_times.append(time.time() - start_time)

        if retrieved_results:
            retrieved_answer = retrieved_results[0][1]  # Best retrieved answer
            similarity_score = compute_cosine_similarity(true_answer, retrieved_answer)
        else:
            retrieved_answer = "No retrieved answer"
            similarity_score = 0.0

        # Store results
        results.append({
            "query": query,
            "true_answer": true_answer,
            "retrieved_answer": retrieved_answer,
            "cosine_similarity": similarity_score
        })

        # Check correctness
        if true_answer.lower() in retrieved_answer.lower():
            correct += 1

    # Compute performance metrics
    avg_latency = np.mean(query_times)
    throughput = len(test_queries) / sum(query_times)
    accuracy = correct / len(test_queries)

    performance_metrics = {
        'average_latency': avg_latency,
        'throughput': throughput,
        'accuracy': accuracy,
        'total_queries': len(test_queries)
    }

    return results, performance_metrics

# Step 9: Define Test Queries and Answers
test_queries = [
    "What is Stable Diffusion?",
    "Who developed Stable Diffusion?",
    "What are the primary uses of Stable Diffusion?",
    "What is the architecture of Stable Diffusion?",
    "What is the diffusion process in Stable Diffusion?",
    "What is the role of latent space in Stable Diffusion?",
    "What is U-Net in Stable Diffusion?",
    "What is the role of CLIP in Stable Diffusion?",
    "How does the decoding process work in Stable Diffusion?",
    "What are the key components of Stable Diffusion?",
    "What is the purpose of text encoding in Stable Diffusion?",
    "What is the training process of Stable Diffusion?",
    "What are some real-world applications of Stable Diffusion?",
    "How is Stable Diffusion used in gaming and virtual worlds?",
    "How can Stable Diffusion assist visually impaired individuals?",
    "What is the role of Variational Autoencoder (VAE) in Stable Diffusion?",
    "What industries benefit from Stable Diffusion?",
    "How does Stable Diffusion handle conditional inputs?",
    "What is the significance of inpainting and outpainting in Stable Diffusion?",
    "How does Stable Diffusion contribute to creative content generation?"
    "What is Newton's First Law of Motion?",
    "What is the formula for kinetic energy?",
    "What is the speed of light in a vacuum?",
    "What is Ohm’s Law?",
    "What is the difference between mass and weight?",
    "What is the second law of thermodynamics?",
    "What is Schrödinger's cat thought experiment?",
    "What is the unit of electric charge?",
    "What is the law of universal gravitation?",
    "What is the principle of superposition in physics?"
]

true_answers = [
    "Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.",
    "Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.",
    "Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided by text prompts.",
    "Stable Diffusion is based on latent diffusion models and consists of components like the diffusion process, latent space, U-Net architecture, and CLIP for conditional inputs.",
    "The diffusion process involves a forward step adding noise until an image becomes pure noise and a reverse step removing noise to generate meaningful images.",
    "Stable Diffusion operates in a lower-dimensional latent space by encoding input images using a variational autoencoder (VAE).",
    "U-Net is a deep neural network in Stable Diffusion that helps denoise latent representations by predicting and removing noise.",
    "CLIP (Contrastive Language–Image Pretraining) encodes user-provided text into embeddings to guide image generation.",
    "The decoding process involves using a VAE decoder to transform the denoised latent representation back into pixel space, generating a high-resolution image.",
    "The key components of Stable Diffusion include the diffusion process, latent space, U-Net architecture, CLIP text encoder, and the VAE for encoding and decoding.",
    "Text encoding in Stable Diffusion converts user-provided text into embeddings using CLIP to guide the image generation process.",
    "Training involves teaching the model to reverse the noise process using large datasets of paired image-text data and optimizing it using loss functions.",
    "Stable Diffusion is used in creative content generation, advertising, gaming, product design, media, education, and accessibility.",
    "In gaming and virtual worlds, Stable Diffusion generates textures, assets, and environments for video games.",
    "Stable Diffusion assists visually impaired individuals by generating images based on textual descriptions.",
    "The Variational Autoencoder (VAE) in Stable Diffusion compresses images into a lower-dimensional latent space and decodes them back into pixel space.",
    "Industries such as advertising, gaming, media, healthcare, and education benefit from Stable Diffusion.",
    "Stable Diffusion processes conditional inputs like text or images to guide image generation using CLIP and other techniques.",
    "Inpainting and outpainting in Stable Diffusion allow modifying parts of an image or extending an image beyond its original boundaries.",
    "Stable Diffusion enables creative content generation by producing artwork, illustrations, and concept designs based on text prompts."
    "Newton's First Law states that an object at rest stays at rest and an object in motion stays in motion unless acted upon by an external force.",
    "The formula for kinetic energy is KE = (1/2)mv², where m is mass and v is velocity.",
    "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s).",
    "Ohm’s Law states that the voltage across a conductor is directly proportional to the current through it, given by V = IR, where V is voltage, I is current, and R is resistance.",
    "Mass is the amount of matter in an object, measured in kilograms, while weight is the force exerted by gravity on that mass, measured in newtons.",
    "The second law of thermodynamics states that the total entropy of an isolated system always increases over time.",
    "Schrödinger's cat is a thought experiment in quantum mechanics where a cat in a box can be both alive and dead until observed, illustrating quantum superposition.",
    "The unit of electric charge is the coulomb (C).",
    "Newton's Law of Universal Gravitation states that every mass attracts every other mass with a force proportional to the product of their masses and inversely proportional to the square of the distance between them: F = G(m₁m₂)/r².",
    "The principle of superposition states that when two or more waves overlap, the resultant displacement at any point is the sum of the displacements due to each individual wave."
]

# Step 10: Run Evaluation and Print Results
evaluation_results, performance_metrics = evaluate_performance_and_similarity(test_queries, true_answers)

# Print Performance Metrics
print("\n🔹 Performance Metrics:")
print(performance_metrics)

# Print Evaluation Results
print("\n🔹 Evaluation Results:")
for result in evaluation_results:
    print(f"🔹 Query: {result['query']}")
    print(f"✔ True Answer: {result['true_answer']}")
    print(f"📌 Retrieved Answer: {result['retrieved_answer']}")
    print(f"🔹 Cosine Similarity: {result['cosine_similarity']:.4f}\n")


{'name': 'instance-0000000000', 'cluster_name': '5d4a93738eab41cca63b4adc57c788c7', 'cluster_uuid': '1yRlIPL7RTm6Z1UZIfjfUg', 'version': {'number': '8.17.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c', 'build_date': '2025-02-05T22:10:57.067596412Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
✅ Successfully connected to Elasticsearch!


Device set to use cpu


✅ Indexed 60 Q&A pairs into Elasticsearch!

🔹 Performance Metrics:
{'average_latency': 0.0405581572960163, 'throughput': 24.655952505471, 'accuracy': 0.0, 'total_queries': 29}

🔹 Evaluation Results:
🔹 Query: What is Stable Diffusion?
✔ True Answer: Stable Diffusion is a deep learning, text-to-image model released in 2022 based on diffusion techniques.
📌 Retrieved Answer:  Stable Diffusion  is a deep learning , text-to-image model  released
🔹 Cosine Similarity: 0.9567

🔹 Query: Who developed Stable Diffusion?
✔ True Answer: Stable Diffusion was developed by Stability AI in collaboration with the CompVis Group at Ludwig Maximilian University of Munich and Runway.
📌 Retrieved Answer:  Stable Diffusion  is a deep learning , text-to-image model  released
🔹 Cosine Similarity: 0.4866

🔹 Query: What are the primary uses of Stable Diffusion?
✔ True Answer: Stable Diffusion is used for generating images based on text descriptions, inpainting, outpainting, and image-to-image translations guided